In [2]:
# 安装依赖（首次运行时需要）
!pip install google-play-scraper pandas

# 引入必要库
import pandas as pd
import time
from google_play_scraper import app, Sort, reviews
from datetime import datetime

# 评论抓取函数
def scrape_app_reviews(app_id, country='us', language='en', count=5000):
    print(f"开始抓取 {app_id} 的评论...")

    app_info = app(
        app_id,
        lang=language,
        country=country
    )
    print(f"应用名称: {app_info['title']}")
    print(f"开发者: {app_info['developer']}")
    print(f"评分: {app_info['score']}")

    results = []
    continuation_token = None
    batch_size = 100

    while len(results) < count:
        current_batch_size = min(batch_size, count - len(results))
        print(f"抓取第 {len(results) + 1} 到 {len(results) + current_batch_size} 条评论...")

        result, continuation_token = reviews(
            app_id,
            lang=language,
            country=country,
            sort=Sort.MOST_RELEVANT,
            count=current_batch_size,
            continuation_token=continuation_token
        )

        if not result:
            print("没有更多评论可抓取")
            break

        results.extend(result)
        time.sleep(1)

    print(f"成功抓取 {len(results)} 条评论")
    df = pd.DataFrame(results)

    if not df.empty:
        def convert_timestamp(ts):
            if isinstance(ts, (int, float)):
                return datetime.fromtimestamp(ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
            else:
                try:
                    return pd.to_datetime(ts).strftime('%Y-%m-%d %H:%M:%S')
                except:
                    return str(ts)

        if 'at' in df.columns:
            df['at'] = df['at'].apply(convert_timestamp)
        if 'repliedAt' in df.columns:
            df['repliedAt'] = df['repliedAt'].apply(lambda x: convert_timestamp(x) if pd.notnull(x) else None)

        df = df.rename(columns={
            'reviewId': '评论ID',
            'userName': '用户名',
            'userImage': '用户头像',
            'content': '评论内容',
            'score': '评分',
            'thumbsUpCount': '点赞数',
            'at': '评论时间',
            'replyContent': '开发者回复',
            'repliedAt': '回复时间'
        })

        columns_to_keep = []
        for col in ['评论ID', '用户名', '评论内容', '评分', '点赞数', '评论时间', '开发者回复', '回复时间']:
            if col in df.columns:
                columns_to_keep.append(col)

        df = df[columns_to_keep]

    return df

# 保存到CSV
def save_to_csv(df, filename):
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"数据已保存到 {filename}")

# 主函数
def main():
    taobao_app_id = "com.taobao.taobao"

    targets = [
        {'country': 'us', 'language': 'en', 'filename': 'taobao_reviews_us_en.csv'},
        {'country': 'cn', 'language': 'zh', 'filename': 'taobao_reviews_cn_zh.csv'}
    ]

    for target in targets:
        try:
            print(f"\n开始处理 {target['country']}/{target['language']} 的评论...")

            df = scrape_app_reviews(
                taobao_app_id,
                country=target['country'],
                language=target['language']
            )

            if not df.empty:
                save_to_csv(df, target['filename'])
                print(f"成功保存 {target['country']}/{target['language']} 的评论数据")
            else:
                print(f"未找到 {target['country']}/{target['language']} 的评论")

        except Exception as e:
            print(f"抓取 {target['country']}/{target['language']} 的评论时出错: {str(e)}")
            import traceback
            traceback.print_exc()

    print("\n所有数据抓取完成!")

# 调用主函数
main()



开始处理 us/en 的评论...
开始抓取 com.taobao.taobao 的评论...
应用名称: Taobao
开发者: Taobao
评分: 2.3874173
抓取第 1 到 100 条评论...
抓取第 101 到 200 条评论...
抓取第 201 到 300 条评论...
抓取第 301 到 400 条评论...
抓取第 401 到 500 条评论...
抓取第 501 到 600 条评论...
抓取第 601 到 700 条评论...
抓取第 701 到 800 条评论...
抓取第 801 到 900 条评论...
抓取第 901 到 1000 条评论...
抓取第 1001 到 1100 条评论...
抓取第 1101 到 1200 条评论...
抓取第 1201 到 1300 条评论...
抓取第 1301 到 1400 条评论...
抓取第 1401 到 1500 条评论...
抓取第 1501 到 1600 条评论...
抓取第 1601 到 1700 条评论...
抓取第 1701 到 1800 条评论...
抓取第 1801 到 1900 条评论...
抓取第 1901 到 2000 条评论...
抓取第 2001 到 2100 条评论...
抓取第 2101 到 2200 条评论...
抓取第 2201 到 2300 条评论...
抓取第 2301 到 2400 条评论...
抓取第 2401 到 2500 条评论...
抓取第 2501 到 2600 条评论...
抓取第 2601 到 2700 条评论...
抓取第 2701 到 2800 条评论...
抓取第 2801 到 2900 条评论...
抓取第 2901 到 3000 条评论...
抓取第 3001 到 3100 条评论...
抓取第 3101 到 3200 条评论...
抓取第 3201 到 3300 条评论...
抓取第 3301 到 3400 条评论...
抓取第 3401 到 3500 条评论...
抓取第 3501 到 3600 条评论...
抓取第 3601 到 3700 条评论...
抓取第 3701 到 3800 条评论...
抓取第 3801 到 3900 条评论...
抓取第 3901 到 4000 条评论...
抓取第 4001 到 41